# Amazon Sagemaker- Use customized algorithms

## 1. Introdiction

In the following project we used amazon sagemaker to have some prediction. To be able to do that, SVM algorithm is used for the classification. Two modeles are created. To get better accurecy StandardScaler is used. First the input data will be preprocssed by the StandardScalar model and after that it will be passed to the SVM model. A Pipiline is used to deploy both of models into one.

## 2. Implementation

In this module, we create a script and deploy it to the amazon sagemaker endpoints.

### Reading the data

In this step, data is uploaded on the notebook instance. we are gonna access to it and upload it to the Amazon S3.

In [6]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import sagemaker

prefix = 'Scikit-SVM-pipeline-thyroid-example'

bucket = sagemaker.Session().default_bucket()

WORK_DIRECTORY = 'thyroid_data'

sagemaker_session = sagemaker.Session()

train_input = sagemaker_session.upload_data(
    path='{}/{}'.format(WORK_DIRECTORY, 'train.csv'), 
    bucket=bucket,
    key_prefix='{}/{}'.format(prefix, 'train'))

- Creating StandardScalar model

In this step, the path to the script is defined and pass it as an entry point to the SKLearn. After configuraing the model we fit it with our input data

In [11]:
sagemaker_session = sagemaker.Session()

script_path = 'sklearn_thyroid_featurizer.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=get_execution_role(),
    train_instance_type="ml.m5.xlarge",
    sagemaker_session=sagemaker_session)
sklearn_preprocessor.fit({'train': train_input})

2019-11-30 19:56:43 Starting - Starting the training job...
2019-11-30 19:56:44 Starting - Launching requested ML instances...
2019-11-30 19:57:41 Starting - Preparing the instances for training......
2019-11-30 19:58:25 Downloading - Downloading input data...
2019-11-30 19:59:14 Training - Training image download completed. Training in progress.
2019-11-30 19:59:14 Uploading - Uploading generated training model
2019-11-30 19:59:14 Completed - Training job completed
2019-11-30 19:59:04,027 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2019-11-30 19:59:04,030 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-30 19:59:04,039 sagemaker_sklearn_container.training INFO     Invoking user training script.
2019-11-30 19:59:04,284 sagemaker-containers INFO     Module sklearn_thyroid_featurizer does not provide a setup.py. 
Generating setup.py
2019-11-30 19:59:04,284 sagemaker-containers INFO     Generating setup.cfg
2019

### Creating the SVM model

In [7]:
sagemaker_session = sagemaker.Session()

script_path = 'sklearn_thyroid_SVM.py'

sklearn_SVM_model = SKLearn(
    entry_point=script_path,
    role=get_execution_role(),
    train_instance_type="ml.m5.xlarge",
    sagemaker_session=sagemaker_session)

sklearn_SVM_model.fit({'train': train_input})

2019-11-30 19:44:17 Starting - Starting the training job...
2019-11-30 19:44:18 Starting - Launching requested ML instances...
2019-11-30 19:45:12 Starting - Preparing the instances for training......
2019-11-30 19:46:08 Downloading - Downloading input data
2019-11-30 19:46:08 Training - Downloading the training image..2019-11-30 19:46:21,723 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2019-11-30 19:46:21,727 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-30 19:46:21,740 sagemaker_sklearn_container.training INFO     Invoking user training script.
2019-11-30 19:46:21,953 sagemaker-containers INFO     Module sklearn_thyroid_SVM does not provide a setup.py. 
Generating setup.py
2019-11-30 19:46:21,953 sagemaker-containers INFO     Generating setup.cfg
2019-11-30 19:46:21,953 sagemaker-containers INFO     Generating MANIFEST.in
2019-11-30 19:46:21,953 sagemaker-containers INFO     Installing module with the fol

### Creatimg the pipline and deploy it to endpiont

In [12]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
scikit_learn_SVM_model = sklearn_SVM_model.create_model()

model_name = 'inference-pipeline-SVM'
sm_model = PipelineModel(
    name=model_name, 
    role=get_execution_role(), 
    models=[
        scikit_learn_inferencee_model, 
        scikit_learn_SVM_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name='thyroid-splitted-final')

--------------------------------------------------------------------------------------!

# 2. Test the model

We deployed the test on the test dataset. The dataset is splitted and 70% is used for the training the model and 30% is used for the test model.

In [16]:
import io
import pandas as pd
import json
import numpy as np
import boto3
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics

data = pd.read_csv('./thyroid_data/test.csv')

runtime= boto3.client('runtime.sagemaker')

features = data.iloc[:, 0:-1]
target = data.iloc[:, -1]

X= np.array(features).astype('float32')
Y= np.array(target).astype('float32')

temp=[]

for x in X:
    response = runtime.invoke_endpoint(EndpointName='thyroid-splitted-final',
                                   ContentType='text/csv',
                                   Body=[",".join(x.astype(str))][0])

    result = json.loads(response['Body'].read().decode())
    test_pred = result['instances'][0]['features']
    temp.append(test_pred)

Y_predict =pd.Series(np.array(temp))

precison = precision_score(Y, Y_predict)
recall = recall_score(Y, Y_predict)
f1 = f1_score(Y, Y_predict)
accuracy = metrics.accuracy_score(Y, Y_predict)

print("accurecy: "+ str(accuracy))
print("precison: "+str(precison))
print("recall: "+ str(recall))
print("f1: "+str(f1))

accurecy: 0.9629629629629629
precison: 0.8395061728395061
recall: 0.7555555555555555
f1: 0.7953216374269005
